In [45]:
import pypdf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

from pathlib import Path

from utils_ccd import get_connection

conn = get_connection()

In [5]:
sql_processos = '''
WITH tab AS (
	SELECT 
		l2.IdProcesso
		,l2.[numero_processo]
		,l2.[ano_processo]
		,p.codigo_tipo_processo
		,p.assunto
		,p.interessado
		,rel.nome as relator
		,l1.[IdLote]
		,l1.[origem]
		,l1.[destino]
		,l1.[enviado_por]
		,l1.[enviado_em]
		,l1.[hora_envio]
		,l2.recebido_em

	  FROM [processo].[dbo].[Lotes] AS l1
	  LEFT JOIN [processo].[dbo].[Itens_Lote] AS l2 ON l1.IdLote = l2.IdLote
	  LEFT JOIN Processo.dbo.Processos AS p ON l2.IdProcesso = p.IdProcesso
	  LEFT JOIN Processo.dbo.Relator AS rel ON p.codigo_relator = rel.codigo
	  WHERE origem = 'ccd'
)

SELECT DISTINCT 
	CONCAT(numero_processo, '/', ano_processo) as processo,
	interessado as jurisdicionado,
	relator,
	assunto,
	FORMAT(recebido_em, 'd', 'pt-BR') as data_recebimento,
	FORMAT(enviado_em, 'd', 'pt-BR') as data_saida
	
	FROM tab
		WHERE YEAR(recebido_em) = 2025 AND MONTH(recebido_em) = 4
'''

In [6]:
processos_ccd = pd.read_sql(sql_processos, conn)

/tmp/ipykernel_67962/2491845865.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  processos_ccd = pd.read_sql(sql_processos, conn)


In [7]:
processos_ccd

,processo,jurisdicionado,relator,assunto,data_recebimento,data_saida
0,000064/2025,CAM.MUN.AREIA BRANCA ...,ANTONIO GILBERTO DE OLIVEIRA JALES,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,29/04/2025,28/04/2025
1,000066/2025,JUNTA COMERCIAL DO ESTADO DO RN ...,ANTONIO ED SOUZA SANTANA,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,29/04/2025,28/04/2025
2,000080/2025,PREFEITURA MUNICIPAL DE ITAÚ ...,PAULO ROBERTO CHAVES ALVES,ENCAMINHA DOCUMENTAÇÃO COMPLEMENTAR REF. A NOT...,24/04/2025,24/04/2025
3,000086/2022,CÂMARA MUNICIPAL DE BODÓ ...,GEORGE MONTENEGRO SOARES,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,29/04/2025,28/04/2025
4,000159/2025,COMPANHIA DE ÁGUAS E ESGOTOS DO RN - CAERN ...,ANTONIO ED SOUZA SANTANA,RESPOSTA À COMUNICAÇÃO TIPO NOTIFICAÇÃO Nº 002...,15/04/2025,15/04/2025
...,...,...,...,...,...,...
235,702547/2012,PREF.MUN.PASSAGEM ...,GEORGE MONTENEGRO SOARES,PRESTAÇÃO DE CONTAS DE ACORDO COM A RESOLUÇÃO ...,24/04/2025,24/04/2025
236,702710/2011,PREF.MUN.LAGOA DE VELHOS ...,RENATO COSTA DIAS,PRESTAÇÃO DE CONTAS DE ACORDO COM A RESOLUÇÃO ...,24/04/2025,24/04/2025
237,703232/2011,PREF.MUN.LAGOA DE VELHOS ...,RENATO COSTA DIAS,PRESTAÇÃO DE CONTAS DE ACORDO COM A RESOLUÇÃO ...,24/04/2025,24/04/2025
238,703336/2012,PREF.MUN.PASSAGEM ...,GEORGE MONTENEGRO SOARES,PRESTAÇÃO DE CONTAS DE ACORDO COM A RESOLUÇÃO ...,24/04/2025,24/04/2025


In [15]:
sql_informacoes_processos = '''
SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
*
FROM processo.dbo.vw_ata_informacao inf
WHERE CONCAT(numero_processo, '/', ano_processo) IN ({})
'''.format(
    ', '.join(["'{}'".format(p) for p in processos_ccd.processo.unique()])
)
informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)

/tmp/ipykernel_67962/4171043796.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)


In [47]:
DIR_INFORMACOES = '/media/informacoes_pdf/'
def get_file_path(row):
    return Path(DIR_INFORMACOES) / row['setor'].strip() / row['arquivo']

In [50]:
informacoes_processos['caminho_arquivo'] = informacoes_processos.apply(get_file_path, axis=1)

In [51]:
informacoes_ccd = informacoes_processos[informacoes_processos['setor'] == 'CCD']

In [52]:
len(processos_ccd)

240

In [53]:
informacoes_ccd['processo'] = informacoes_ccd['numero_processo'] + '/' + informacoes_ccd['ano_processo'].astype(str)

/tmp/ipykernel_67962/3746687965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  informacoes_ccd['processo'] = informacoes_ccd['numero_processo'] + '/' + informacoes_ccd['ano_processo'].astype(str)


In [54]:
# get last information for each process by informacoes_ccd['DataPublicacao']

informacoes_ccd_last = informacoes_ccd.sort_values(by='DataPublicacao').drop_duplicates(subset='processo', keep='last')

In [55]:
len(informacoes_ccd_last), len(informacoes_ccd), len(processos_ccd)

(26, 33, 240)

In [56]:
processos_ccd[~processos_ccd['processo'].isin(informacoes_ccd_last['processo'])]['processo']

2      000080/2025
4      000159/2025
5      000340/2025
6      000422/2021
7      000441/2018
          ...     
235    702547/2012
236    702710/2011
237    703232/2011
238    703336/2012
239    997660/2012
Name: processo, Length: 214, dtype: object

In [57]:
(3718-3463)*.96

244.79999999999998

In [58]:
# put all texts from informacoes_ccd_last['caminho_arquivo'] into a string column 'texto'
def extract_text_from_pdf(file_path):
    try:
        with open(file_path, 'rb') as f:
            reader = pypdf.PdfReader(f)
            text = ''
            for page in reader.pages:
                text += page.extract_text() or ''
            return text
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ''
informacoes_ccd_last['texto'] = informacoes_ccd_last['caminho_arquivo'].apply(extract_text_from_pdf)

In [61]:
print(informacoes_ccd_last['texto'].iloc[0])

CANCELAMENTO DE PARCELAMENTO
Processo n°: 200069/2021
Código da Dívida: 24.534
Responsável(eis)
Nome: GIVAGNO PATRESE DA SILVA BEZERRA (VEREADOR CPF/CNPJ: 07088099446
CERTIFICO, para fins do disposto no artigo 20, § 3º, da Resolução nº 13/2015-TCE/RN, que o (s) 
responsável(eis) acima identificado (s) não efetuou (ram) o pagamento da parcela 3/20, com vencimento em 
31/12/2024 no valor de 473,46, referente à multa identificada pelo código 24.534, devida ao Fundo de 
Reaparelhamento e Aperfeiçoamento (FRAP), constituída nos autos do processo nº 200069/2021.
CERTIFICO, ainda, que o parcelamento deferido anteriormente foi cancelado , vencendo-se 
antecipadamente as parcelas restantes.
Data da Emissão do Cancelamento do Parcelamento: 09/01/2025
Jumara Sasaki
Assistente Técnico de Diretoria de Controle Externo - CC4
Matrícula: 10.050-1
______________________________________________________________________
Av. Presidente Getúlio Vargas, 690 - Ed. Dr. Múcio Vilar Ribeiro Dantas
CEP 59012-360 

In [63]:
def get_descricao_objeto(texto):
    pass

In [64]:
def get_descricao_encaminhamento(texto):
    pass